In [ ]:
!pip install rouge_score pycocoevalcap==1.2

In [2]:
import tensorflow as tf

# Check for GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU is enabled and available!")
    for gpu in gpus:
        print(f"Device: {gpu}")
else:
    print("GPU is not available. Check your Kaggle settings.")

GPU is enabled and available!
Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Device: PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [4]:
# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Enable mixed precision training
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

Num GPUs Available:  2


In [ ]:
import os
import torch
import numpy as np

# Set random seeds for reproducibility
torch.manual_seed(7580)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(7580)

# Set the dataset paths
image_dir = '/kaggle/input/iu-xray/iu_xray/images'  # updated path as per new values
ann_path = '/kaggle/input/iu-xray/annotation.json'  # updated path as per new values

# Make sure the required directories exist
os.makedirs('/kaggle/working/results/iu_xray', exist_ok=True)

# Import the necessary modules (ensure they are uploaded to Colab or in the appropriate paths)
import sys
sys.path.append('/kaggle/input/r2gencmn')  # Update this with the actual path if needed

# Import functions and classes
from models.models import BaseCMNModel
from modules.dataloaders import R2DataLoader
from modules.loss import compute_loss
from modules.metrics import compute_scores
from modules.optimizers import build_optimizer, build_lr_scheduler
from modules.tokenizers import Tokenizer
from modules.trainer import Trainer

# Define the args manually (equivalent to argparse in the original script)
class Args:
    def __init__(self):
        self.image_dir = '/kaggle/input/iu-xray/iu_xray/images'  # updated default
        self.ann_path = '/kaggle/input/iu-xray/iu_xray/annotation.json'  # updated default
        self.dataset_name = 'iu_xray'  # updated default
        self.max_seq_length = 60  # updated default
        self.threshold = 3  # updated default
        self.num_workers =  2  # default value (unchanged)
        self.batch_size = 8  # updated default
        self.visual_extractor = 'resnet101'  # default value (unchanged)
        self.visual_extractor_pretrained = True  # default value (unchanged)
        self.d_model = 512  # default value (unchanged)
        self.d_ff = 512  # default value (unchanged)
        self.d_vf = 2048  # default value (unchanged)
        self.num_heads = 8  # default value (unchanged)
        self.num_layers = 3  # updated default
        self.dropout = 0.1  # default value (unchanged)
        self.logit_layers = 1  # default value (unchanged)
        self.bos_idx = 0  # default value (unchanged)
        self.eos_idx = 0  # default value (unchanged)
        self.pad_idx = 0  # default value (unchanged)
        self.use_bn = 0  # default value (unchanged)
        self.drop_prob_lm = 0.5  # default value (unchanged)
        self.topk = 32  # updated default
        self.cmm_size = 2048  # updated default
        self.cmm_dim = 512  # updated default
        self.sample_method = 'beam_search'  # default value (unchanged)
        self.beam_size = 3  # updated default
        self.temperature = 1.0  # default value (unchanged)
        self.sample_n = 1  # default value (unchanged)
        self.group_size = 1  # default value (unchanged)
        self.output_logsoftmax = 1  # default value (unchanged)
        self.decoding_constraint = 0  # default value (unchanged)
        self.block_trigrams = 1  # default value (unchanged)
        self.n_gpu = 1  # default value (unchanged)
        self.epochs = 100  # updated default
        self.save_dir = '/kaggle/working/results/iu_xray'  # updated default
        self.resume = None  # default value (unchanged)
        self.record_dir = 'records/'  # default value (unchanged)
        self.log_period = 50  # updated default
        self.save_period = 1  # default value (unchanged)
        self.monitor_mode = 'max'  # default value (unchanged)
        self.monitor_metric = 'BLEU_4'  # default value (unchanged)
        self.early_stop = 50  # default value (unchanged)
        self.optim = 'Adam'  # default value (unchanged)
        self.lr_ve = 1e-4  # updated default
        self.lr_ed = 5e-4  # updated default
        self.step_size = 10  # updated default
        self.gamma = 0.8  # updated default
        self.adam_betas = (0.9, 0.98)  # default value (unchanged)
        self.adam_eps = 1e-9  # default value (unchanged)
        self.amsgrad = True  # default value (unchanged)
        self.noamopt_warmup = 5000  # default value (unchanged)
        self.noamopt_factor = 1  # default value (unchanged)
        self.lr_scheduler = 'StepLR'  # default value (unchanged)
        self.seed = 7580  # updated default
        self.weight_decay= 5e-5

args = Args()

# Create tokenizer
tokenizer = Tokenizer(args)

# Data loader setup
train_dataloader = R2DataLoader(args, tokenizer, split='train', shuffle=True)
val_dataloader = R2DataLoader(args, tokenizer, split='val', shuffle=False)
test_dataloader = R2DataLoader(args, tokenizer, split='test', shuffle=False)

# Build model architecture
model = BaseCMNModel(args, tokenizer)

# Set up the loss function and metrics
criterion = compute_loss
metrics = compute_scores

# Set up the optimizer and learning rate scheduler
optimizer = build_optimizer(args, model)
lr_scheduler = build_lr_scheduler(args, optimizer)

# Specify the mode you want to run:
mode = 'train'  # Change to 'test' or 'eval' based on what you need to do

# Create trainer and start training/evaluating
trainer = Trainer(model, criterion, metrics, optimizer, args, lr_scheduler, train_dataloader, val_dataloader, test_dataloader)

# If you're training, you would call:
if mode == 'train':
    trainer.train()

# If you're testing, you would call:
elif mode == 'test':
    trainer.test()
s

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithr

	epoch          : 1
	train_loss     : 2.73326284084541
	val_BLEU_1     : 0.1988625889127359
	val_BLEU_2     : 0.1180353903908086
	val_BLEU_3     : 0.0835544869652126
	val_BLEU_4     : 0.06400886298630389
	val_METEOR     : 0.1034722429799017
	val_ROUGE_L    : 0.29474007494202537
	test_BLEU_1    : 0.2414891728040812
	test_BLEU_2    : 0.14657508017251872
	test_BLEU_3    : 0.10545486468993152
	test_BLEU_4    : 0.08101704657824713
	test_METEOR    : 0.11945841070409312
	test_ROUGE_L   : 0.3121804906222145
Saving checkpoint: /kaggle/working/results/iu_xray/current_checkpoint.pth ...
Saving current best: model_best.pth ...


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	epoch          : 2
	train_loss     : 1.8735506668053998
	val_BLEU_1     : 0.24932250524655303
	val_BLEU_2     : 0.11406621701689343
	val_BLEU_3     : 0.06253531594430263
	val_BLEU_4     : 0.03631005422436742
	val_METEOR     : 0.09201997556791261
	val_ROUGE_L    : 0.24651998446111154
	test_BLEU_1    : 0.29558595794337766
	test_BLEU_2    : 0.1582887552272947
	test_BLEU_3    : 0.10431492558089035
	test_BLEU_4    : 0.07552251751913235
	test_METEOR    : 0.11082379996356859
	test_ROUGE_L   : 0.2874023707795102
Saving checkpoint: /kaggle/working/results/iu_xray/current_checkpoint.pth ...


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


...

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	epoch          : 40
	train_loss     : 0.2830807818063898
	val_BLEU_1     : 0.34324554891823167
	val_BLEU_2     : 0.19269806790944802
	val_BLEU_3     : 0.11982806871238377
	val_BLEU_4     : 0.07855777449583021
	val_METEOR     : 0.13950458694694917
	val_ROUGE_L    : 0.285819267376419
	test_BLEU_1    : 0.37003248036104236
	test_BLEU_2    : 0.21734365797931163
	test_BLEU_3    : 0.14613074283384017
	test_BLEU_4    : 0.10678705486273188
	test_METEOR    : 0.15020932136785245
	test_ROUGE_L   : 0.3138296740356642
Saving checkpoint: /kaggle/working/results/iu_xray/current_checkpoint.pth ...


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


	epoch          : 41
	train_loss     : 0.27119464953663724
	val_BLEU_1     : 0.37511505043099114
	val_BLEU_2     : 0.2261058116215141
	val_BLEU_3     : 0.15481278723839906
	val_BLEU_4     : 0.11366328572216977
	val_METEOR     : 0.15017843033276124
	val_ROUGE_L    : 0.3029360768161515
	test_BLEU_1    : 0.3929351552949671
	test_BLEU_2    : 0.2374970915071431
	test_BLEU_3    : 0.16259448032851095
	test_BLEU_4    : 0.1186497720819163
	test_METEOR    : 0.1575622254496484
	test_ROUGE_L   : 0.30965091971243086
Saving checkpoint: /kaggle/working/results/iu_xray/current_checkpoint.pth ...


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
import pycocoevalcap.bleu
print(dir(pycocoevalcap.bleu))
